# 實價登錄

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from time import sleep
import os
import re
import requests as rq


options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("--incognito")
options.add_argument("--disable-popup-blocking")

driver = webdriver.Chrome( options = options )

# Visit 實價登錄 open data
def visit():
    driver.get("https://plvr.land.moi.gov.tw/DownloadOpenData");
    sleep(2)

# Selection for dataset
def select():
    
    # 非本期下載
    select_past = driver.find_element(By.CSS_SELECTOR, "a#ui-id-2")
    select_past.click()
    sleep(2)
    
    # 檔案格式 -- csv
    select_csv = Select(driver.find_element(By.CSS_SELECTOR, "select#fileFormatId"))
    select_csv.select_by_value("csv")
    sleep(2)
    
    # 下載方式 -- 進階
    select_type = driver.find_element(By.CSS_SELECTOR, "#downloadTypeId2")
    select_type.click()
    sleep(2)
    
    # 新北市3個選項
    select_city1 = driver.find_element(By.CSS_SELECTOR, "#table5 > tbody > tr:nth-child(8) > td:nth-child(2) > input")
    select_city1.click()
    sleep(1)
    
    select_city2 = driver.find_element(By.CSS_SELECTOR, "#table5 > tbody > tr:nth-child(8) > td:nth-child(3) > input")
    select_city2.click()
    sleep(1)
    
    select_city3 = driver.find_element(By.CSS_SELECTOR, "#table5 > tbody > tr:nth-child(8) > td:nth-child(4) > input")
    select_city3.click()
    sleep(1)
    
    # 發布日期
    select_season = Select(driver.find_element_by_name("season"))
    opt = select_season.options
    for index in range(0, len(opt)):
        select_season.select_by_index(index)
        sleep(1)
        # click download
        select_download = select_type = driver.find_element(By.CSS_SELECTOR, "#downloadBtnId")
        select_download.click()
        sleep(15)
    

def close():
    driver.quit()  
    
if __name__ == '__main__':
    visit()
    select()
    close()

# Bus stop

In [ ]:
import numpy as np
import pandas as pd
import json
import csv

In [ ]:
for i in range(0, 33):
    busURL = "https://data.ntpc.gov.tw/api/datasets/34B402A8-53D9-483D-9406-24A682C2D6DC/csv?page=" + str(i) + "&size=1500"
    
    busreq = rq.get(busURL)
    url_con = busreq.content
    with open("./data/bus/" + str(i) + ".csv", "wb") as f:
        f.write(url_con)

# 周邊設施

In [ ]:
# 讀取整理後(有包含房屋物件的經緯度)的大表
df1 = pd.read_csv("./combine/presale_location.csv")

In [ ]:
# 抓出不重複的地號，存成list
df2 = df1.drop_duplicates(subset = ["土地區段位置建物區段門牌"])
add = list(df2["土地區段位置建物區段門牌"])

# 在整理出地號各自的經緯度，存成list
x_h = list(df2["x_h"])
y_h = list(df2["y_h"])

## 嫌惡設施

In [ ]:
# 嫌惡設施 API
# 500m以內有嫌惡設施
h_dic1 = {}
for i in range(0, len(x_h)):
    addr = add[i]
    badURL = "https://api.nlsc.gov.tw/other/MarkBufferAnlys/dis/" + str(x_h[i]) + "/" + str(y_h[i]) + "/500"
    bad_h = rq.get(badURL).json()
    h_dic1[addr] = bad_h
    
# 100m以內有嫌惡設施
h_dic2 = {}
for i in range(0, len(x_h)):
    addr = add[i]
    badURL = "https://api.nlsc.gov.tw/other/MarkBufferAnlys/dis/" + str(x_h[i]) + "/" + str(y_h[i]) + "/100"
    bad_h = rq.get(badURL).json()
    h_dic2[addr] = bad_h

In [ ]:
# check 哪幾個地方附近在100m有嫌惡設施
badList1 = []

for i in range(0, len(x_h)):
    if len(list(h_dic2.values())[i]) != 0:
        badList1.append(list(h_dic2.keys())[i])

badList1

In [ ]:
# 確認有無殯儀館、機場、墓...在500公尺內
h_list1 = []
for i in range(0, len(x_h)):
    badURL = "https://api.nlsc.gov.tw/other/MarkBufferAnlys/dis/" + str(x_h[i]) + "/" + str(y_h[i]) + "/500"
    bad_h = rq.get(badURL).json()
    h_list3.append(bad_h)

word = "殯儀" # 可替換成其他字詞
for a in h_list1:
    for b in a:
        if word in b["name"]:
            print("YES")
        else:
            print("NO")

In [ ]:
# 存出去(100m, 500m)
with open("./data/spot/bad5m.json", "w", encoding = "utf-8") as a:
    a.write(json.dumps(h_dic1, ensure_ascii = False))
    
with open("./data/spot/bad1m.json", "w", encoding = "utf-8") as b:
    b.write(json.dumps(h_dic2, ensure_ascii = False))

## 文教設施

In [ ]:
# 文教設施 API
# 100m
h_dic1 = {}
for i in range(0, len(x_h)):
    addr = add[i]
    eduURL = "https://api.nlsc.gov.tw/other/MarkBufferAnlys/edu/" + str(x_h[i]) + "/" + str(y_h[i]) + "/100"
    edu_h = rq.get(eduURL).json()
    h_dic1[addr] = edu_h
    
# 500m
h_dic2 = {}
for i in range(0, len(x_h)):
    addr = add[i]
    eduURL = "https://api.nlsc.gov.tw/other/MarkBufferAnlys/edu/" + str(x_h[i]) + "/" + str(y_h[i]) + "/500"
    edu_h = rq.get(eduURL).json()
    h_dic2[addr] = edu_h
    
# 1000m
h_dic3 = {}
for i in range(0, len(x_h)):
    addr = add[i]
    eduURL = "https://api.nlsc.gov.tw/other/MarkBufferAnlys/edu/" + str(x_h[i]) + "/" + str(y_h[i]) + "/1000"
    edu_h = rq.get(eduURL).json()
    h_dic3[addr] = edu_h

In [ ]:
# 存出去(1000m, 500m)
with open("./data/spot/edu10m.json", "w", encoding = "utf-8") as a:
    a.write(json.dumps(h_dic3, ensure_ascii = False))
    
with open("./data/spot/edu5m.json", "w", encoding = "utf-8") as b:
    b.write(json.dumps(h_dic2, ensure_ascii = False))

## 醫療設施

In [ ]:
# 醫療設施 API 
# 100m
h_dic4 = {}
for i in range(0, len(x_h)):
    addr = add[i]
    medURL = "https://api.nlsc.gov.tw/other/MarkBufferAnlys/med/" + str(x_h[i]) + "/" + str(y_h[i]) + "/100"
    med_h = rq.get(medURL).json()
    h_dic4[addr] = med_h

# 500m
h_dic5 = {}
for i in range(0, len(x_h)):
    addr = add[i]
    medURL = "https://api.nlsc.gov.tw/other/MarkBufferAnlys/med/" + str(x_h[i]) + "/" + str(y_h[i]) + "/500"
    med_h = rq.get(medURL).json()
    h_dic5[addr] = med_h
    
# 1000m
h_dic6 = {}
for i in range(0, len(x_h)):
    addr = add[i]
    medURL = "https://api.nlsc.gov.tw/other/MarkBufferAnlys/med/" + str(x_h[i]) + "/" + str(y_h[i]) + "/1000"
    med_h = rq.get(medURL).json()
    h_dic6[addr] = med_h

In [ ]:
# 存出去
with open("./data/spot/med1m.json", "w", encoding = "utf-8") as a:
    a.write(json.dumps(h_dic4, ensure_ascii = False))
    
with open("./data/spot/med5m.json", "w", encoding = "utf-8") as b:
    b.write(json.dumps(h_dic5, ensure_ascii = False))

with open("./data/spot/med10m.json", "w", encoding = "utf-8") as c:
    c.write(json.dumps(h_dic6, ensure_ascii = False))